In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_cluster 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH d_store AS
(
    SELECT * EXCEPT(ds.region_domain),
            CASE WHEN ds.region_domain = 'West' THEN 'South' ELSE ds.region_domain END AS region_domain
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store ds
)
SELECT
    store_id,
    product_id,
    cluster_sale AS cluster,
    start_date,
    end_date
FROM (
    SELECT
        a.*,
        ROW_NUMBER() OVER (PARTITION BY store_id,product_id,end_date ORDER BY proc_date) rn
    FROM (
        SELECT
            store_id,product_id,cluster_sale,proc_date,start_date,end_date
        FROM {catalog_name}.udp_wcm_silver_dim.stg_vm_danhmucnhomhangmn
        UNION ALL 
        SELECT
            store_id, product_id, cluster_sale, proc_date, start_date,end_date
        FROM {catalog_name}.udp_wcm_silver_dim.stg_vm_danhmucnhomhangmb
        UNION ALL 
        SELECT b.store_id, product_id,CLUSTER cluster_sale, proc_date,start_date,end_date
        FROM 
            {catalog_name}.udp_wcm_silver_dim.stg_vmp_assortmentclustering a
            LEFT JOIN d_store b ON b.region_domain = a.region AND business_unit = '2000'
    ) a
) x WHERE x.rn = 1
""")